In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
sys.path.insert(0, '../src')
from scipy import stats
import rff, subsample, herding
%matplotlib inline

ImportError: No module named rff